# Assignment 2 Cox Regression Analysis with Time-Varying Covariates 

*Due February 28*


In this assignment you will learn to estimate the risks associated with covariates that change over time, such as RNA measurements. The basics of Cox regression are in Chapter 3 of “Applied Survival Analysis”. 

## Conceptual Questions

**1)** What are the advantages and disadvantages of Kaplan-Meier vs Cox Regression Analysis?

| |Advantages|Disadvantages|
|--|--|--|
|Kaplan-Meier|Simple model to implement.|Cannot use multiple parameters. Cannot handle hazard.|
|Cox Regression|Can use multiple parameters. Can handle hazard.| |



**2)** What additional implementation issues arise when implementing time-varying covariates in Cox regression vs. baseline covariates only?

*reference Time-varying covariates in the documentation to the python package “lifelines” https://lifelines.readthedocs.io/en/latest/Time%20varying%20survival%20regression.html*

## Coding part:

Here we will implement a function for performing Cox regression, and then extend it to time-varying covariates.

### Step 1) Test data simulation

**1a)** Pick a set of test beta-coefficients (= log Hazard Ratios)
for a set of 2-3 “predictors” with data matrix X. You can generate the entries of X with a standard normal distribution.


In [1]:
using DataFrames

df_test = DataFrame(
    Event = [0, 1, 1, 1],
    Time = [2, 4, 4, 8], 
    betaX = 1:4
)

,Event,Time,betaX
,Int64,Int64,Int64
1,0,2,1
2,1,4,2
3,1,4,3
4,1,8,4


From article
T = 100
time = [1:101]
k = [1, 30, 35, 39, 44, 60, 71, 80, 95, 101]  [1:10]
heights = [0.0000000, 0.1099288, 0.2112084, 0.2702198, 0.3528798, 0.4038134, 0.4548563, 0.5743559, 0.7256261, 1.0000000] [1:10]

tk = 
| |time|height|
|-|-|-|
|1|1|0.0000000|
|2|30|0.1099288|
|3|35|0.2112084|
|4|39|0.2702198|
|5|44|0.3528798|
|6|60|0.4038134|
|7|71|0.4548563|
|8|80|0.5743559|
|9|95|0.7256261|
|10|101|1.0000000|

bl.failure.CDF
  [1] 0.000000e+00 4.507311e-06 3.605849e-05 1.216974e-04 2.884679e-04
  [6] 5.634138e-04 9.735791e-04 1.546008e-03 2.307743e-03 3.285830e-03
 [11] 4.507311e-03 5.999231e-03 7.788633e-03 9.902562e-03 1.236806e-02
 [16] 1.521217e-02 1.846194e-02 2.214442e-02 2.628664e-02 3.091564e-02
 [21] 3.605849e-02 4.174220e-02 4.799384e-02 5.484045e-02 6.230906e-02
 [26] 7.042673e-02 7.922049e-02 8.871740e-02 9.894448e-02 1.099288e-01
 [31] 1.251110e-01 1.458804e-01 1.691866e-01 1.919793e-01 2.112084e-01
 [36] 2.267433e-01 2.411218e-01 2.552964e-01 2.702198e-01 2.872822e-01
 [41] 3.059750e-01 3.244027e-01 3.406695e-01 3.528798e-01 3.618454e-01
 [46] 3.696919e-01 3.764938e-01 3.823258e-01 3.872625e-01 3.913784e-01
 [51] 3.947483e-01 3.974467e-01 3.995482e-01 4.011275e-01 4.022590e-01
 [56] 4.030176e-01 4.034777e-01 4.037139e-01 4.038010e-01 4.038134e-01
 [61] 4.040392e-01 4.047951e-01 4.061987e-01 4.083675e-01 4.114193e-01
 [66] 4.154715e-01 4.206418e-01 4.270478e-01 4.348072e-01 4.440374e-01
 [71] 4.548563e-01 4.672661e-01 4.809026e-01 4.953330e-01 5.101246e-01
 [76] 5.248448e-01 5.390607e-01 5.523396e-01 5.642489e-01 5.743559e-01
 [81] 5.823789e-01 5.886411e-01 5.936169e-01 5.977805e-01 6.016061e-01
 [86] 6.055682e-01 6.101411e-01 6.157989e-01 6.230161e-01 6.322669e-01
 [91] 6.440257e-01 6.587667e-01 6.769642e-01 6.990925e-01 7.256261e-01
 [96] 7.570210e-01 7.936614e-01 8.359135e-01 8.841432e-01 9.387167e-01
[101] 1.000000e+00



In [22]:
# constants
T = 100
knots = 8 
time = 1:(T+1)

# random vars - must be changed
k = [1, 30, 35, 39, 44, 60, 71, 80, 95, 101]
heights = [0.0000000, 0.1099288, 0.2112084, 0.2702198, 0.3528798, 0.4038134, 0.4548563, 0.5743559, 0.7256261, 1.0000000]

10-element Vector{Float64}:
 0.0
 0.1099288
 0.2112084
 0.2702198
 0.3528798
 0.4038134
 0.4548563
 0.5743559
 0.7256261
 1.0

In [23]:
using DataFrames

tk = DataFrame(time = k, heights = heights)
#tk[:, :heights]

,time,heights
,Int64,Float64
1,1,0.0
2,30,0.109929
3,35,0.211208
4,39,0.27022
5,44,0.35288
6,60,0.403813
7,71,0.454856
8,80,0.574356
9,95,0.725626


In [24]:
using Plots
plotlyjs(size = (570, 570))

xs = tk[:, :time]
ys = tk[:, :heights]

itp = Interpolator(xs, ys)

x_range = range(xs[1], stop=xs[end], length=100)
y_interpolated = [itp(x) for x in x_range]

trace_org = scatter(x=xs, y=ys, mode="markers")

trace_itp = scatter(x=x_range, y=y_interpolated, mode="lines")

plot([trace_org, trace_itp], Layout(yaxis_title="Cumulative P(Failure)", xaxis_title="Time", title="Cubic spline (failure CDF)"))

data: [
  "scatter with fields mode, type, x, and y",
  "scatter with fields mode, type, x, and y"
]

layout: "layout with fields margin, template, title, xaxis, and yaxis"

In [27]:
baseline_failure_CDF = [itp(x) for x in time]

101-element Vector{Float64}:
 0.0
 0.003970881938471204
 0.007802795097939828
 0.011509750027956834
 0.01510575727807326
 0.01860482739784012
 0.02202097093680841
 0.025368198444529155
 0.02866052047055336
 0.03191194756443201
 0.03513649027571616
 0.038348159153956785
 0.04156096474870494
 ⋮
 0.6653489601073157
 0.6753825180086599
 0.6862587967369036
 0.6981460105037433
 0.7112123735208766
 0.7256261
 0.7493009722848667
 0.7872915829373278
 0.8354773614962274
 0.8897377375004097
 0.9459521404887193
 1.0

In [39]:
baseline = DataFrame(time = time[1:(T)],
                    failure_PDF = diff(baseline_failure_CDF),
                    failure_CDF = baseline_failure_CDF[2:end],
                    survivor = 1 .- baseline_failure_CDF[2:end]
)

,time,failure_PDF,failure_CDF,survivor
,Int64,Float64,Float64,Float64
1,1,0.00397088,0.00397088,0.996029
2,2,0.00383191,0.0078028,0.992197
3,3,0.00370695,0.0115098,0.98849
4,4,0.00359601,0.0151058,0.984894
5,5,0.00349907,0.0186048,0.981395
6,6,0.00341614,0.022021,0.977979
7,7,0.00334723,0.0253682,0.974632
8,8,0.00329232,0.0286605,0.971339
9,9,0.00325143,0.0319119,0.968088


In [44]:
baseline[!, :hazard] = baseline[:, :failure_PDF] ./ (1 .- baseline_failure_CDF[1:(T)])
@show baseline

baseline = 100×5 DataFrame
 Row │ time   failure_PDF  failure_CDF  survivor   hazard
     │ Int64  Float64      Float64      Float64    Float64
─────┼────────────────────────────────────────────────────────
   1 │     1   0.00397088   0.00397088  0.996029   0.00397088
   2 │     2   0.00383191   0.0078028   0.992197   0.00384719
   3 │     3   0.00370695   0.0115098   0.98849    0.00373611
   4 │     4   0.00359601   0.0151058   0.984894   0.00363788
   5 │     5   0.00349907   0.0186048   0.981395   0.00355274
   6 │     6   0.00341614   0.022021    0.977979   0.00348091
   7 │     7   0.00334723   0.0253682   0.974632   0.0034226
   8 │     8   0.00329232   0.0286605   0.971339   0.00337802
   9 │     9   0.00325143   0.0319119   0.968088   0.00334736
  10 │    10   0.00322454   0.0351365   0.964864   0.00333084
  11 │    11   0.00321167   0.0383482   0.961652   0.00332863
  12 │    12   0.00321281   0.041561    0.958439   0.00334092
  13 │    13   0.00322795   0.0447889   0.955211  

,time,failure_PDF,failure_CDF,survivor,hazard
,Int64,Float64,Float64,Float64,Float64
1,1,0.00397088,0.00397088,0.996029,0.00397088
2,2,0.00383191,0.0078028,0.992197,0.00384719
3,3,0.00370695,0.0115098,0.98849,0.00373611
4,4,0.00359601,0.0151058,0.984894,0.00363788
5,5,0.00349907,0.0186048,0.981395,0.00355274
6,6,0.00341614,0.022021,0.977979,0.00348091
7,7,0.00334723,0.0253682,0.974632,0.0034226
8,8,0.00329232,0.0286605,0.971339,0.00337802
9,9,0.00325143,0.0319119,0.968088,0.00334736


In [45]:
d1 = DataFrame(knots = k, heights = heights)
d2 = DataFrame(time = 1:T,
               cdf = baseline[:, :failure_CDF],
               pdf = baseline[:, :failure_PDF],
               survivor = baseline[:, :survivor],
               hazard = baseline[:, :hazard])

,time,cdf,pdf,survivor,hazard
,Int64,Float64,Float64,Float64,Float64
1,1,0.00397088,0.00397088,0.996029,0.00397088
2,2,0.0078028,0.00383191,0.992197,0.00384719
3,3,0.0115098,0.00370695,0.98849,0.00373611
4,4,0.0151058,0.00359601,0.984894,0.00363788
5,5,0.0186048,0.00349907,0.981395,0.00355274
6,6,0.022021,0.00341614,0.977979,0.00348091
7,7,0.0253682,0.00334723,0.974632,0.0034226
8,8,0.0286605,0.00329232,0.971339,0.00337802
9,9,0.0319119,0.00325143,0.968088,0.00334736


**1b)** Create a random failure CDF according to the method of Harden and Kropko (see Generating the Baseline Hazard Function). Plot your baseline survival function (1 - failure CDF).

In [3]:
using Distributions
using PlotlyJS
using DataFrames


# create a uniform vector between 1 and 10
T_max = 100
k = 10
c = zeros(1)

append!( c, sort(rand(Uniform(0, 9), 8)), k)

xs = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90]

ys = c / 10

plot(scatter(x=xs, y=ys, mode="markers"),
     Layout(yaxis_title="Cumulative P(Failure)", xaxis_title="Time", title="Random unifor distrubuted points"))

WebIO._IJuliaInit()

data: [
  "scatter with fields mode, type, x, and y"
]

layout: "layout with fields margin, template, title, xaxis, and yaxis"

In [4]:
# Construct failure CDF by fitting a cubic smoothing spline - Hymans cubic smoothing function
using PCHIPInterpolation

println(length(xs))
println(length(ys))

itp = Interpolator(xs, ys)

x_range = range(xs[1], stop=xs[end], length=100)
y_interpolated = [itp(x) for x in x_range]

trace_org = scatter(x=xs, y=ys, mode="markers")

trace_itp = scatter(x=x_range, y=y_interpolated, mode="lines")

plot([trace_org, trace_itp], Layout(yaxis_title="Cumulative P(Failure)", xaxis_title="Time", title="Cubic spline (failure CDF)"))


10
10


data: [
  "scatter with fields mode, type, x, and y",
  "scatter with fields mode, type, x, and y"
]

layout: "layout with fields margin, template, title, xaxis, and yaxis"

**1c)** For each subject in your study, generate a synthetic survival time (see Harden and Kropko Generating Durations). We can assume for now that there is no censoring. Note that a survival function is a CDF for the event-time. This means that you can use simple inverse sampling to get survival time samples. 

In [5]:
# get PDF of failure CDF by using the gradient for the failure CDF
using Interpolations
using ShiftedArrays
#using TimeSeries
#using Pandas

df = DataFrame(failure_cdf=y_interpolated)
df = transform(df, :failure_cdf => (x -> x - lag(x)) => :failure_pdf)
df = transform(df, :failure_cdf => (x -> (1 .- x)) => :survivor)
println(df)



100×3 DataFrame
 Row │ failure_cdf  failure_pdf        survivor  
     │ Float64      Float64?           Float64   
─────┼───────────────────────────────────────────
   1 │ 0.0          missing            1.0
   2 │ 0.000316531        0.000316531  0.999683
   3 │ 0.00123628         0.000919747  0.998764
   4 │ 0.00271447         0.00147819   0.997286
   5 │ 0.00470633         0.00199186   0.995294
   6 │ 0.00716709         0.00246076   0.992833
   7 │ 0.010052           0.00288489   0.989948
   8 │ 0.0133162          0.00326425   0.986684
   9 │ 0.0169151          0.00359884   0.983085
  10 │ 0.0208037          0.00388865   0.979196
  11 │ 0.0249374          0.00413369   0.975063
  12 │ 0.0292714          0.00433397   0.970729
  13 │ 0.0355036          0.00623224   0.964496
  14 │ 0.0449497          0.00944605   0.95505
  15 │ 0.0569911          0.0120414    0.943009
  16 │ 0.0710093          0.0140183    0.928991
  17 │ 0.086386           0.0153767    0.913614
  18 │ 0.102503         

### Step 2) Cox model implementation

**2a)** Create a test function which calls your to be implemented Cox model with your test survival data. You should assert that the estimated hazard ratio values are reasonably close to the ones you used to create your test data.

**2b)** Implement a Cox regression model that estimates hazard ratios based on event-time data (duration, censoring). You can use a standard optimization package to perform likelihood maximisation. A Newton-type method such as L-BFGS-B or SQP should be sufficient. The likelihood function should hopefully be calculated numerically by the optimization routine. Otherwise you can get it by finite-differencing or algorithmic differentiation. 

**2c)** Test your new Cox Regression model function with your test function.

### Step 3) Extension

**3a)** Add a random censoring mechanism to your test data. You will need to pick a distribution function for censoring that depends on time. Example functions are uniform (simple) or exponential (to cluster the failure time at the beginning or end of the interval).  An easy way to control the amount of censoring is to extend the censoring PDF to times past the limit of your survival PDF. Check that your Cox model still estimates reasonable hazard ratios given a large enough synthetic sample size.

**3b)** Create a new test data function and cox regression model for time-varying covariates. You can copy and update your previous functions.
You will need to allow your data X to vary over time. This can be achieved with a stochastic process like the Ornstein-Uhlenbeck model, which allows us to control the amount of time-variation in each covariate.

https://ipython-books.github.io/134-simulating-a-stochastic-differential-equation/

To get the subject specific survival function you can numerically integrate the baseline hazard function multiplied by (X(t) dot beta) over time to get a hazard function. The survival function is then exp(-hazard).
